In [184]:
import os
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

# Setup Python path to include the project root
project_dir = Path.cwd().parent
if str(project_dir) not in sys.path:
    sys.path.insert(0, str(project_dir))

# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import the agent and state management class
from ProductDesigner.agents.safe_product_interviewer import SafeProductInterviewerAgent, logging
from ProductDesigner.graph_state import GraphState

# Imports neccessary to run the cells below
from pydantic import BaseModel
from typing import List, Dict, Any
from langchain.output_parsers import PydanticOutputParser
from openai import AsyncOpenAI
import asyncio
import os

print('Setup complete. Necessary packages and paths are loaded.')

Setup complete. Necessary packages and paths are loaded.


# 1. Agent Initialization

In [2]:
# Define the initial product idea and create a mock state
initial_idea = "An AI-powered mutual fund recommendation system for indian MFs - fetching data from public or 3rd part API, produce vizualizations, track investments, suggest switches, analyze performanc eo fmultuaal funds, determine best MFs for a given risk type, optimal allocation"
mock_state = {"initial_idea": initial_idea, "qna_history": {}}

# Initialize the interviewer agent
interviewer = SafeProductInterviewerAgent()
print("Agent initialized successfully.")

questions = interviewer._get_planning_questions()
questions

2025-08-07 23:39:55,051 - INFO - LLMService initialized with model: gpt-4o
2025-08-07 23:39:55,372 - INFO - Web search is enabled.


Agent initialized successfully.


['What is the primary purpose of this application or system?',
 'Who are the main users or target audience of this application?',
 'What are the 3-5 core features needed for the MVP (Minimum Viable Product)?',
 'What specific problem does this solution solve for your users?',
 'Are there any existing solutions to this problem? How is yours different?',
 'What technologies or tech stack do you have in mind? Or would you like recommendations?',
 'What are the main user flows or journeys through the application?',
 'Are there any specific UI/UX requirements or preferences?',
 'What are your plans for data storage and management?',
 'Are there any specific security requirements or concerns?',
 'What is your timeline for development of the MVP?',
 'Are there any third-party integrations needed?',
 'What metrics would define success for this product?']

## Agent 1: Get the Answers of the Questions I have prepared previously

In [ ]:
class QA(BaseModel):
    question: str
    answer: str

class QALIST(BaseModel):
    questions: List[QA]

    def str(self):
        return '\n'.join([f"Question: {qa.question}\nAnswer: {qa.answer}" for qa in self.questions])

parser = PydanticOutputParser(pydantic_object=QALIST)
format_instructions = parser.get_format_instructions()

questions = interviewer._get_planning_questions()
str_questions = '\n'.join(questions)
template = (
            "You are an expert product manager. Based on the initial idea and any relevant search results, "
            "provide a comprehensive and insightful answer to the following questions. Each answer will be displayed to the client for helping them decide their answer. So make each answer as informative as possible with recommendations, options, industry standards, examples of existing products with simiiar focus etc.\n\n"
            "Initial Idea: {initial_idea}\n"
            "Questions: \n {questions}\n"
            "Include reasoning, multiple options where applicable, and industry best practices.\n"
            "Answer Format Instructions: "
            "\n{format_instructions}"
        )

params = {"questions": str_questions, 
            "initial_idea": initial_idea,
            "format_instructions": format_instructions}
initial_qa = interviewer._run_llm_step(dict(), template, params, "Could not generate an initial answer due to an error.", output_parser=parser, model_override="gpt-4.1", temparature_override=0.1)

2025-07-31 01:40:50,479 - INFO - Executing LLM chain with params: dict_keys(['questions', 'initial_idea', 'format_instructions'])
2025-07-31 01:41:10,330 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-31 01:41:10,347 - INFO - LLM chain executed successfully.


## Agent 2: Create Web search from the answers for enriching the Answers Further

In [10]:
class Websearch(BaseModel):
    search_queries: List[str]

parser = PydanticOutputParser(pydantic_object=Websearch)
format_instructions = parser.get_format_instructions()

template = (
            "You are an expert product manager. Based on the initial idea and the initial QA with the client, generate a list of search queries to perform web search, to enrich the answers"
            "Each answer will be displayed to the client for helping them decide their answer. So make the queries to extract relevant and timely information. Aim to enrich the recommendations, options, industry standards, examples of existing products with simiiar focus etc.\n\n"
            "Initial Idea: {initial_idea}\n"
            "QA: \n {initial_qa}\n"
            "Include reasoning, multiple options where applicable, and industry best practices.\n"
            "Answer Format Instructions: "
            "\n{format_instructions}"
        )

params = {"initial_idea": initial_idea,
            "format_instructions": format_instructions,
            "initial_qa": initial_qa.str()}

websearch = interviewer._run_llm_step(dict(), template, params, "Could not generate the list of web search queries.", output_parser=parser, model_override="gpt-4.1", temparature_override=0.1)

2025-07-31 01:52:18,155 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'format_instructions', 'initial_qa'])
2025-07-31 01:52:23,207 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-31 01:52:23,220 - INFO - LLM chain executed successfully.


## Create the Agent Network

In [120]:
from pydantic import Field


class Agent(BaseModel):
    name:str
    responsibility:str
    parent:str
    children:List[str]
    input:List[str] = Field(default_factory=list, description="list of fields from the state that will act as the input for this agent")
    output:List[str] = Field(default_factory=list, description="list of fields from the state that will act as the output for this agent")

class State(BaseModel):
    property_names: List[str] = Field(default_factory=list, description="list of properties of the langraph state for the agentic network")
    property_descriptions: List[str]
    property_defaults: List[str]

class Graph(BaseModel):
    agents: List[Agent]
    state: State
    
parser = PydanticOutputParser(pydantic_object=Graph)
format_instructions = parser.get_format_instructions()

template = (
            "Given the initial idea of the product, design an Langraph based agentic network to produce documents neccessary for an LLM based coding assistant to execute the product idea autonomously."
            "Human intervention should be minimal - every human interaction should have a default which the human can simply accept or provide feedback. "
            "The documents produced should be in markdown format. "
            "The agentic network should be able to discover information and perform web search to enrich the answers. Use that information to make decisions about the architecture as well as low level implementaation detials."
            "The documentation should meet professional standards but targetted at beginners. \n\n"
            "Initial Idea: {initial_idea}\n"
            "Answer Format Instructions: "
            "\n{format_instructions}"
        )

params = {"initial_idea": initial_idea,
            "format_instructions": format_instructions}

#websearch = interviewer._run_llm_step(dict(), template, params, "Could not generate the list of web search queries.", output_parser=parser, model_override="gpt-5")
# Merge the template and params to create teh full prompt
full_prompt = template.format(**params)

# Call openai
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
            
response = await client.responses.create(
    model="o3",
    input=full_prompt,
    tools=[
        {
            "type": "web_search_preview"
        }
    ]
)

content = response.output_text
# extract json from content


2025-08-09 05:13:55,675 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


In [128]:
graph = parser._parse_obj(json.loads(content))
graph

Graph(agents=[Agent(name='OrchestratorAgent', responsibility='Root supervisor that sequences all child-agents, resolves dependencies, applies default human choices when no feedback is received, and merges every markdown output into “final_docs”.', parent='root', children=['ProductManagerAgent', 'ArchitectureAgent', 'DevOpsAgent', 'DocumentationAgent'], input=['idea', 'user_feedback'], output=['final_docs']), Agent(name='ProductManagerAgent', responsibility='Transform the raw idea into a beginner-friendly Product-Requirements-Document (PRD) that covers goals, user stories, MVP scope, personas, and compliance assumptions.  Defaults to current idea if the user supplies no extra info.', parent='OrchestratorAgent', children=['ResearchAgent'], input=['idea', 'user_feedback'], output=['prd']), Agent(name='ResearchAgent', responsibility='Perform live web search to discover Indian mutual-fund data sources (e.g., AMFI daily NAV files, mftool library, BSE-Star/NSE-NMF order APIs, 3rd-party projec

## Agent 1: Generating a Knowledgebase                                                                                                                                                                                                                                       

In [3]:
from ProductDesigner.agents.research_subgraph_agent import ResearchSubgraphAgent, ResearchState, _execute_parallel_searches_for_queries

agent = ResearchSubgraphAgent()
research_state = ResearchState(
                initial_idea=initial_idea,
                max_depth=5
            )

2025-08-07 23:40:00,406 - INFO - LLMService initialized with model: gpt-4o


### Graph

In [4]:
research_state = agent._generate_queries_node(research_state)

2025-08-07 23:40:07,274 - INFO - Generating initial search queries...
2025-08-07 23:40:07,310 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'format_instructions'])
2025-08-07 23:40:14,881 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-07 23:40:14,965 - INFO - LLM chain executed successfully.
2025-08-07 23:40:14,967 - INFO - Generated 30 search queries
2025-08-07 23:40:14,968 - INFO - 1. Existing AI-powered mutual fund recommendation platforms in India
2025-08-07 23:40:14,968 - INFO - 2. Popular mutual fund analysis tools for Indian investors
2025-08-07 23:40:14,971 - INFO - 3. Feature comparison of Indian mutual fund tracking apps
2025-08-07 23:40:14,971 - INFO - 4. User journey for mutual fund investment and tracking platforms
2025-08-07 23:40:14,971 - INFO - 5. Tech stack used by leading mutual fund recommendation systems
2025-08-07 23:40:14,973 - INFO - 6. Open source codebases for mutual fund analysis on Githu

In [5]:
research_state =  agent._execute_parallel_search_node(research_state)

2025-08-07 23:40:15,028 - INFO - ---1 ---
2025-08-07 23:40:15,029 - INFO - --- Existing AI-powered mutual fund recommendation platforms in India ---
2025-08-07 23:40:15,031 - INFO - --- openai_web_search ---
2025-08-07 23:40:15,032 - INFO - --- Popular mutual fund analysis tools for Indian investors ---
2025-08-07 23:40:15,035 - INFO - --- openai_web_search ---
2025-08-07 23:40:15,037 - INFO - --- Feature comparison of Indian mutual fund tracking apps ---
2025-08-07 23:40:15,039 - INFO - --- openai_web_search ---
2025-08-07 23:40:15,040 - INFO - --- User journey for mutual fund investment and tracking platforms ---
2025-08-07 23:40:15,042 - INFO - --- openai_web_search ---
2025-08-07 23:40:15,043 - INFO - --- Tech stack used by leading mutual fund recommendation systems ---
2025-08-07 23:40:15,045 - INFO - --- openai_web_search ---
2025-08-07 23:40:15,046 - INFO - --- Open source codebases for mutual fund analysis on Github ---
2025-08-07 23:40:15,046 - INFO - --- openai_web_search ---

In [6]:
research_state = agent._evaluate_learnings_node(research_state)
agent._should_continue_research(research_state)

2025-08-07 23:40:47,788 - INFO - Evaluating learnings at depth 2
2025-08-07 23:40:47,791 - INFO - Total learnings so far: 30


'continue'

In [7]:
research_state = agent._generate_queries_node(research_state)


2025-08-07 23:40:47,818 - INFO - Evaluating current learnings for additional search needs...
2025-08-07 23:40:47,822 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'format_instructions', 'topics'])
2025-08-07 23:40:50,215 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-07 23:40:50,222 - INFO - LLM chain executed successfully.
2025-08-07 23:40:50,223 - INFO - No additional queries needed - research is sufficient.
2025-08-07 23:40:50,226 - INFO - Generated 0 queries for depth 2


In [8]:
research_state = agent._execute_parallel_search_node(research_state)
research_state = agent._evaluate_learnings_node(research_state)
agent._should_continue_research(research_state)


2025-08-07 23:40:50,249 - INFO - No queries to execute
2025-08-07 23:40:50,258 - INFO - Evaluating learnings at depth 3
2025-08-07 23:40:50,259 - INFO - Total learnings so far: 30


'finish'

In [9]:
research_state = agent._generate_knowledge_base_node(research_state)

2025-08-07 23:40:50,278 - INFO - Generating knowledge base from research learnings...
2025-08-07 23:40:50,281 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'research_findings'])
2025-08-07 23:41:32,089 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-07 23:41:32,152 - INFO - LLM chain executed successfully.
2025-08-07 23:41:32,158 - INFO - Knowledge base generation completed
2025-08-07 23:41:32,159 - INFO - Knowledge base generation completed


### Inidividual Agent Scratch

In [10]:
learning_texts = [learning.combined_content for learning in research_state.learnings]
learning_text = "\n\n========\n\n".join(learning_texts)
template = (
            "You are a consultant the User has hired to help them build a product. Based on the initial product idea and all the research findings from your research agents is provided below, "
            "The client wants to build the product - your primary goal is to enable them with comprehensive knowledge base that will help them make informed decisions."
            "The client will use this knowledge base to make decisions about how to execute the product idea. Focus on execution and implementation details. Prevent reinventing the wheel wherever possible by leveraging existing open source libraries. "
            "Give sufficient details of all the libraries your suggest. If there is external data involved - make sure to give sufficient details of the data sources, data models, api etc.."
            "Here are some suggestions for the knowledge base (use this as a seed, you are not restricted by the following list and feel free to add or remove any section):"
            "1. Executive Summary "
            "2. Existing Solutions and Features we can leverage for user product idea"
            "3. Technical Architecture & Stack Recommendations "
            "4. Feature Analysis (Core MVP vs Nice-to-have) "
            "5. User Experience & Journey Insights for the user product idea and those already covered by existing solutions "
            "6. Implementation Resources (Libraries, APIs, Code Examples) "
            "7. Gap Analysis - what additional features do we need to implement to make the user product idea successful"
            "8. Development Recommendations for architecture, functional modules, data models, APIs  "
            "9. UI/UX recommendations for the user product idea"
            "\n"
            "Initial Idea: {initial_idea}\n"
            "Research Findings: \n{research_findings}\n"
            "\n"
            "Provide a well-structured, actionable knowledge base that will help in product development decisions."
        )

params = {
    "initial_idea": initial_idea,
    "research_findings": learning_text,
}

knowledge_base = agent._run_llm_step(
    dict(), template, params,
    "Could not generate knowledge base from research findings.",
    model_override="gpt-4.1",
    temperature_override=0.
)
knowledge_base

2025-08-07 23:46:10,701 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'research_findings'])
2025-08-07 23:47:18,413 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-07 23:47:18,524 - INFO - LLM chain executed successfully.


'# Knowledge Base for Building an AI-Powered Mutual Fund Recommendation System for Indian MFs\n\n---\n\n## 1. Executive Summary\n\nThis document provides a comprehensive knowledge base for developing an AI-powered mutual fund recommendation system tailored to Indian investors. The platform will fetch data from public/third-party APIs, visualize fund performance, track investments, suggest fund switches, analyze performance, recommend optimal funds based on risk profiles, and provide allocation strategies. The goal is to leverage existing open-source libraries and APIs, minimize redundant development, and ensure regulatory compliance and robust security.\n\n---\n\n## 2. Existing Solutions & Features to Leverage\n\n### Notable Platforms in India\n- **Risify AI, InvestorAi, 5nance, ET Money Genius, Jarvis Invest, Fisdom, Wright Research, SBI SmartAssist**: Offer AI-driven recommendations, portfolio insights, risk profiling, and automation.\n- **Value Research Online, RupeeVest, Kuvera, my

## Direct Generation of the Knowledge Base

In [ ]:
import re
import csv
from io import StringIO
from pprint import pprint

class TechStack(BaseModel):
    Layer: str
    Option: str
    WhyFitsMVP: str

class DataSourceAPI(BaseModel):
    Name: str
    Documentation_Link: str
    GettingStarted: str
    LicenceCost: str
    MCPIfAvailable: str

class ThirdPartyComponents(BaseModel):
    Name: str
    Link: str
    Purpose: str
    DocumentationLink: str
    APIReferenceLink: str

class ComparableProducts(BaseModel):
    Name: str
    Link: str
    TakeAway: str

class KnowledgeBase(BaseModel):
    ProductIdeaSummary: str
    CoreMVPFeatures: List[str]
    NiceToHaveFeatures: List[str]
    TechStackRecommendation: List[TechStack]
    DataSourcesAPIsStorageAndManagement: List[DataSourceAPI]
    ReusableComponents: List[ThirdPartyComponents]
    InspirationalProducts: List[ComparableProducts]
    UIUXKitsExamples: List[str]
    EffortvsImpactMatrix: str
    DeploymentPath: str
    CostandTimelineEstimate: str
    KeyUserStories: List[str]
    RisksandMitigations: str
    SourceLinks: str

    def to_markdown(self):
        """Converts the knowledge base into a markdown formatted string."""
        markdown_str = f"# Product Knowledge Base\n\n"

        # Product Idea Summary
        markdown_str += f"## Product Idea Summary\n{self.ProductIdeaSummary}\n\n"

        # Core MVP Features
        markdown_str += "## Core MVP Features\n"
        for feature in self.CoreMVPFeatures:
            markdown_str += f"- {feature}\n"
        markdown_str += "\n"

        # Nice To Have Features
        markdown_str += "## Nice To Have Features\n"
        for feature in self.NiceToHaveFeatures:
            markdown_str += f"- {feature}\n"
        markdown_str += "\n"
        
        # Key User Stories
        markdown_str += "## Key User Stories\n"
        for story in self.KeyUserStories:
            markdown_str += f"- {story}\n"
        markdown_str += "\n"

        # Tech Stack Recommendation
        markdown_str += "## Tech Stack Recommendation\n"
        markdown_str += "| Layer | Option | Why it Fits MVP |\n"
        markdown_str += "|---|---|---|\n"
        for item in self.TechStackRecommendation:
            markdown_str += f"| {item.Layer} | {item.Option} | {item.WhyFitsMVP} |\n"
        markdown_str += "\n"

        # Data Sources, APIs, Storage and Management
        markdown_str += "## Data Sources, APIs, Storage and Management\n"
        markdown_str += "| Name | Documentation | Getting Started | License/Cost | MCP if Available |\n"
        markdown_str += "|---|---|---|---|---|\n"
        for item in self.DataSourcesAPIsStorageAndManagement:
            markdown_str += f"| {item.Name} | [Link]({item.Documentation_Link}) | {item.GettingStarted} | {item.LicenceCost} | {item.MCPIfAvailable} |\n"
        markdown_str += "\n"

        # Reusable Components
        markdown_str += "## Reusable Components\n"
        markdown_str += "| Name | Link | Purpose | Documentation | API Reference |\n"
        markdown_str += "|---|---|---|---|---|\n"
        for item in self.ReusableComponents:
            markdown_str += f"| {item.Name} | [Link]({item.Link}) | {item.Purpose} | [Doc]({item.DocumentationLink}) | [API]({item.APIReferenceLink}) |\n"
        markdown_str += "\n"

        # Inspirational Products
        markdown_str += "## Inspirational Products\n"
        markdown_str += "| Name | Link | Takeaway |\n"
        markdown_str += "|---|---|---|\n"
        for item in self.InspirationalProducts:
            markdown_str += f"| {item.Name} | [Link]({item.Link}) | {item.TakeAway} |\n"
        markdown_str += "\n"
        
        # UI/UX Kits & Examples
        markdown_str += "## UI/UX Kits & Examples\n"
        for item in self.UIUXKitsExamples:
            markdown_str += f"- {item}\n"
        markdown_str += "\n"

        # Effort vs. Impact Matrix
        markdown_str += f"## Effort vs. Impact Matrix\n{self.EffortvsImpactMatrix}\n\n"

        # Deployment Path
        markdown_str += f"## Deployment Path\n{self.DeploymentPath}\n\n"

        # Cost and Timeline Estimate
        markdown_str += f"## Cost and Timeline Estimate\n{self.CostandTimelineEstimate}\n\n"

        # Risks and Mitigations
        markdown_str += f"## Risks and Mitigations\n{self.RisksandMitigations}\n\n"

        # Source Links
        markdown_str += f"## Source Links\n{self.SourceLinks}\n\n"

        return markdown_str


class SectionScore(BaseModel):
    Dimension: str
    Score: int
    Comment: str
    
class HighPriorityIssue(BaseModel):
    Issue: str
    Impact: str
    Fix: str

class ImprovementTask(BaseModel):
    SectionToFix: str
    ExactChange: str
    Why:str

class Critique(BaseModel):
    OverallScore: int
    ScoreBreakdown: List[SectionScore]
    HighPriorityIssues: List[HighPriorityIssue]
    ImprovementTasks: List[ImprovementTask]

    def to_markdown(self):
        """Converts the critique into a markdown formatted string."""
        markdown_str = "## Critique Analysis\n\n"

        markdown_str += "### Strengths\n"
        if self.strengths:
            for item in self.strengths:
                markdown_str += f"- {item}\n"
        else:
            markdown_str += "No specific strengths identified.\n"
        markdown_str += "\n"

        markdown_str += "### Weaknesses\n"
        if self.weaknesses:
            for item in self.weaknesses:
                markdown_str += f"- {item}\n"
        else:
            markdown_str += "No specific weaknesses identified.\n"
        markdown_str += "\n"

        markdown_str += "### Suggestions for Improvement\n"
        if self.suggestions:
            for item in self.suggestions:
                markdown_str += f"- {item}\n"
        else:
            markdown_str += "No specific suggestions provided.\n"
        markdown_str += "\n"

        return markdown_str

class ResearchState(BaseModel):
    initial_idea: str = Field(description="Initial idea for the research")
    curr_knowledge_base: KnowledgeBase = Field(default="", description="Knowledge base from the list of learnings")
    critique: Critique = Field(default="", description="Critique of the knowledge base")
    knowledge_bases: List[KnowledgeBase] = Field(default_factory=list, description="List of knowledge bases")
    messages: List[Dict[str, Any]] = Field(default_factory=list, description="List of messages")



In [ ]:
async def knowledge_base_topics(state: ResearchState) -> ResearchState:
    pass

async def knowledge_base_task(state: ResearchState) -> ResearchState:

    prompt = f"""### System (Agent Role)  
    You are an **Execution-MVP Research Agent**.  
    Your mission is the following, using the information in the "INPUTS" section below - 
    1). Turn a *user-supplied product idea* into a **ready-to-build MVP blueprint**, using **web search** and only reputable public sources. 
    2). Optimise for speed-to-prototype and maximum reuse of existing tech.
    3). Ensure Comprehensiveness of the knowledge base - it should be able to cover all the aspects of the product idea.
    4). Critically evaluate the relevance of each section to the provided product idea. If a section is not applicable, omit it and briefly state why (e.g., "Not applicable because this is a hardware product")
    --------------------------------------------------------------------
    1️⃣ DELIVERABLES • JSON with the following keys (use exact IDs):

    1. ProductIdeaSummary · ≤ 75-word recap in user’s terminology  
    2. CoreMVPFeatures · list of features that are essential for the MVP
    3. NiceToHaveFeatures · bullet list  
    4. TechStackRecommendation · list of dictionaries with keys: Layer | Options | WhyFitsMVP 
    5. DataSourcesAndDataAPIs · List of dictionaries with keys: Name | Documentation_links | Getting_started | MCP
    6. ReusableComponentsAndServices  (OpenSource Only)
      • Name
      • Link
      • Purpose
      • Documentation Link
      • API Reference Link
    7. ComparableProducts · (≤ 5 ) List of dictionaries with keys: Name | Link | Take-away
    8. UIUXKitsExamples · bullet list (links or screenshots + licence)  
    9. EffortvsImpactMatrix · Markdown Table, 2 × 2 table (Quick Win, Big Bet, Fill-Ins, Icebox)  
    10. DeploymentPath · hosting, DB, CI/CD; free-tier notes  
    11. CostandTimelineEstimate · rough ranges for build, infra, third-party fees  
    12. KeyUserStories · List of stories in the form “As a [user], I want [action], so that [benefit].”  
    13. RisksandMitigations · technical / regulatory blockers + mitigations  
    14. SourceLinks · numbered list; cite as [S1], [S2] inline above

    --------------------------------------------------------------------
    2️⃣ RESEARCH METHOD  

    • **Query Design** – start broad, then specialised per deliverable.  
    • **Source Quality Filter** – prefer  
      – Official docs / GitHub / vendor blogs < 18 months old  
      – Authoritative review sites (G2, Gartner, etc.)  
      – Standards or academic sources for compliance issues  
    • **De-duplication** – keep best source when findings overlap.  
    • **Citation** – append [S#] after every factual statement or link.  
    • **Critical Thinking** – flag thin evidence with “⚠ Evidence weak”.

    --------------------------------------------------------------------
    4️⃣ STOP CONDITIONS  

    • Stop when all 13 headers are filled **or** 60 outbound links reached.  
    • If a section is genuinely N/A, write: `N/A – no relevant info found [Sx]`.

    --------------------------------------------------------------------
    5️⃣ DEPTH-OF-INQUIRY RULES  

    For *each execution dimension* (Tech Stack, Inspirational Services,
    Reusable Components, Feature Set, UI/UX, User Stories) do:

    1. **Enumerate** at least N candidates (see table below).  
    2. For **every candidate**, provide  
      – **What it is** (≤ 15 words)  
      – **Why it matters for this MVP** (value prop)  
      – **Quick-start (3 steps)** – actionable integration outline  
      – **Effort / Impact** – integers 1-5 each  
      – **Caveats / Limitations** – licence, region, scaling, etc.  
    3. **Evidence quota** – cite ≥ 3 unique sources per sub-dimension or mark `[Gap]`.  
    4. If quota unmet, list search queries tried and label sub-dimension “Open-Question”.

    | Dimension            | Minimum N |
    |----------------------|-----------|
    | Tech Stack options   | 5 |
    | Inspirational services| 4 |
    | Reusable components  | 6 |
    | Core features        | 5 |
    | UI/UX kits           | 3 |
    | User stories         | 8 |

    *Micro-format example (Tech Stack option)*  
    ```markdown
    ### Frontend Option 1 – Next.js 14 [S12]  
    - **Why:** React ecosystem, SEO-friendly SSR, fast scaffold.  
    - **Quick-start:**  
      1. `npx create-next-app@latest`  
      2. Add Tailwind → `npm i -D tailwindcss`  
      3. Deploy free to Vercel → `vc deploy`  
    - **Effort / Impact:** 2 / 5  
    - **Caveats:** File routing unfamiliar to Angular devs.  
    - **Sources:** [S12] [S13] [S15]  

    INPUTS: 
    ---------------------------
    User Supplied Product Idea: {state.initial_idea} 
    
    """
    client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    state.messages.append({"role": "user", "content": prompt})
    response = await client.responses.parse(
        model="gpt-4.1",
        input=state.messages,
        tools=[
            {
                "type": "web_search_preview"
            }
        ],
        temperature=0.1,
        text_format=KnowledgeBase
    )

    content = response.output_parsed
    state.curr_knowledge_base = content
    state.messages.append({"role": "assistant", "content": content.model_dump_json()})
    return state

async def knowledge_base_critique(state:ResearchState) -> ResearchState:
    critique_conv = """
      > **Role**  
      > You are a **Critique Agent**. Your task is to **assess the quality** of the above knowledge-base and to give **concrete, actionable instructions** for raising its quality to a publish-ready standard.
      > Each Key in the knowledge base is a dimension of the research that needs to be evaluated.
      > Your focus should be on factual correctness, complenetess and actionability of the knowledgebase - don't worry about formatting or grammatical errors.
      > It is not compulsory for you to suggest any improvements if you don't find any issues - you can return empty lists below
      > Score each dimension **1 – 5** (5 = excellent) and justify in ≤ 30 words. 
      > Evaluate the following dimensions as they apply to building the product by an LLM code assistant - to prevent hallucination and achieve determinism.
      > A good knowledge base will have sufficient information for an LLM based code assistant to build the product unsupervised with as little as possible human intervention. 
      > A good knowledge base will have emphasis on log term maintainability and extensibility. The architecture is modular but managable.
      --------------------------------------------------------------------
      3️⃣ OUTPUT FORMAT (JSON with the following keys (use exact IDs))

    1. Overall Score: **NN/50** : NN = Sum of scores of each dimension
    2. Score-Breakdown: List of Dictionaries with keys Dimension | Score | Comment
    3. HighPriorityIssues: (≤ 5): List of Dictionaries with keys Issue | Impact | Fix
    4. Improvement-Instructions: (Atomic Tasks): List of Dictionaries with keys SectionToFix | ExactChange | Why
    """
    state.messages = state.messages + [
        {"role": "user", "content": critique_conv}
    ]
    
    client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    response = await client.responses.parse(
        model="gpt-5",
        input=state.messages,
        tools=[
            {
                "type": "web_search"
            }
        ],
        text_format = Critique
    )

    #content = response.output[0].content[0].text
    content = response.output_parsed
    state.critique = content
    state.messages = state.messages + [
        {"role": "assistant", "content": content.model_dump_json()}
    ]
    return state

async def implement_changes_task(state: ResearchState) -> ResearchState:
    state.messages = state.messages + [{'role':'system', 'content': state.critique.model_dump_json()}, 
    {'role':'user', 'content': "Given the above improvement tasks and high priority issues, can you make the changes in the above knowledge base and return the updated knowledge base?"}
    ]
    client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    response = await client.responses.parse(
        model="gpt-4.1",
        input=state.messages,
        tools=[
            {
                "type": "web_search_preview"
            }
        ],
        temperature=0.1,
        text_format=KnowledgeBase
    )

    kb_mod = response.output_parsed
    state.knowledge_bases.append(state.curr_knowledge_base)
    state.curr_knowledge_base = kb_mod
    return state

def purge_messages(state: ResearchState) -> ResearchState:
    messages = []
    messages.append(state.messages[0])
    state.messages.append({"role": "assistant", "content": state.curr_knowledge_base.model_dump_json()})
    state.messages = messages
    return state

async def generate_knowledge_base(initial_idea:str, num_enrichment_loops:int=2) -> ResearchState:
    MAX_ITER = num_enrichment_loops
    idea = initial_idea

    state = ResearchState(initial_idea = idea)


    logging.info(f"generating knowledge_base")
    # ----- Generate / Revise KB -----
    state = await knowledge_base_task(state)
    logging.info(f"knowledge_base generated")

    for i in range(1, MAX_ITER + 1):
        logging.info(f"iteration {i}")
        
        # ----- Critique -----
        logging.info(f"critiquing knowledge_base")
        state = await knowledge_base_critique(state)
        logging.info(f"knowledge_base critiqued")

        if len(state.critique.ImprovementTasks) > 0:
            # ----- Make Changes -----
            logging.info("Running task to modify the knowledge base")
            state = await implement_changes_task(state)
            logging.info("knowledge base modified")
            state = purge_messages(state)
        else:
            logging.info("No changes needed")
            break

        #modify messages
    else:
        print("⚠ Max iterations reached; manual review recommended.")
    
    return state


In [200]:
knowledge_base = await generate_knowledge_base(initial_idea)

2025-08-12 04:33:33,360 - INFO - generating knowledge_base
2025-08-12 04:33:57,274 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 04:33:57,545 - INFO - knowledge_base generated
2025-08-12 04:33:57,545 - INFO - iteration 1
2025-08-12 04:33:57,547 - INFO - critiquing knowledge_base
2025-08-12 04:34:56,971 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 04:34:56,993 - INFO - knowledge_base critiqued
2025-08-12 04:34:56,995 - INFO - Running task to modify the knowledge base
2025-08-12 04:35:25,458 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 04:35:25,707 - INFO - knowledge base modified
2025-08-12 04:35:25,708 - INFO - iteration 2
2025-08-12 04:35:25,710 - INFO - critiquing knowledge_base
2025-08-12 04:36:17,308 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 04:36:17,345 - INFO - knowledge_base critiqued
202

⚠ Max iterations reached; manual review recommended.


# 2. Stage 1: Web Search for Context

In [ ]:
from langchain.prompts import PromptTemplate

class Websearch(BaseModel):
    search_queries: List[str]

parser = PydanticOutputParser(pydantic_object=Websearch)
format_instructions = parser.get_format_instructions()

template = ("""Role: You are an expert Research Strategist and Senior Product Analyst.

Objective: Your goal is to generate a list of precise, high-impact web search queries to validate, enrich, and expand upon a specific section of a product's Knowledge Base.

Context:

Product Idea: {product_idea}
Section Name: {section_name}
Current Section Content:
{section_content}
Your Task: Based on the provided context, generate a list of 3-5 web search queries. These queries should be strategically designed to find information that will significantly improve the quality, accuracy, and depth of this section.

Guidelines for Generating Queries:

Prioritize Impact: Focus on queries that will yield the most valuable information. Avoid trivial or overly broad searches.
Validate Key Assumptions: Generate queries to verify critical data points, such as pricing, project maintenance status, or best practices (e.g., "[library name]" GitHub issues and last commit date).
Seek Deeper Insights: Create queries to find tutorials, implementation guides, or expert opinions (e.g., tutorial for integrating "[API name]" with [Tech Stack component]).
Explore Alternatives: Formulate queries to discover competing or alternative solutions (e.g., "[component name]" vs [competitor] performance comparison 2024).
Be Specific and Concise: Use keywords from the content. Keep queries short and to the point.

OUTPUT FORMAT:
{format_instructions}
""")

params = {"product_idea": knowledge_base.initial_idea,
            "format_instructions": format_instructions,
            "section_name": "CoreMVPFeatures", 
            "section_content": knowledge_base.curr_knowledge_base.CoreMVPFeatures}

websearch = interviewer._run_llm_step(dict(), template, params, "Could not generate the list of web search queries.", output_parser=parser, model_override="gpt-4.1", temparature_override=0.1)

2025-08-12 05:01:38,102 - INFO - Executing LLM chain with params: dict_keys(['product_idea', 'format_instructions', 'section_name', 'section_content'])
2025-08-12 05:01:40,621 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-12 05:01:40,691 - INFO - LLM chain executed successfully.


In [209]:
websearch.search_queries

['best public APIs for real-time Indian mutual fund data 2024',
 'implementing interactive mutual fund performance charts in React',
 'mutual fund portfolio tracking open source libraries India',
 'algorithm for optimal mutual fund allocation based on risk profile',
 'mutual fund switch recommendation system case studies India']

In [208]:
# Create a search query and fetch context from the web
class SearchResuts(BaseModel):
    search_queries: List[str] = []
    search_results: List[str] = []

    def str(self):
        return '\n'.join([f"Query: {qa[0]}\n Search Results: {qa[1]}" for qa in zip(self.search_queries, self.search_results)])

search_results = SearchResuts()
for query in websearch.search_queries:
    search_result = interviewer.search_web_for_context(query)
    search_results.search_queries.append(query)
    search_results.search_results.append(search_result)

print("--- Web Search Results ---")
print(search_results)

2025-08-12 05:02:58,251 - INFO - Performing web search for: best public APIs for real-time Indian mutual fund data 2024
c:\Code\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
2025-08-12 05:02:58,684 - INFO - response: https://www.bing.com/search?q=best+public+APIs+for+real-time+Indian+mutual+fund+data+2024&filters=ex1%3A%22ez5_19947_20312%22 200
2025-08-12 05:02:58,755 - INFO - Web search successful.
2025-08-12 05:02:58,755 - INFO - Performing web search for: implementing interactive mutual fund performance charts in React
c:\Code\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
2025-08-12 05:02:58,982 - INFO - response: https://www.bing.com/search?q=imp

--- Web Search Results ---
search_queries=['best public APIs for real-time Indian mutual fund data 2024', 'implementing interactive mutual fund performance charts in React', 'mutual fund portfolio tracking open source libraries India', 'algorithm for optimal mutual fund allocation based on risk profile', 'mutual fund switch recommendation system case studies India'] search_results=['No good DuckDuckGo Search Result was found', 'Mar 6, 2015 · My customer wants me to change the packing method and I tell him that I will implement this change in/on/to … May 24, 2020 · Hello, I was just wondering which one of these two prepositions is better in this context. It\'s quite hard to … Nov 6, 2009 · Our customer have three new feature requests. We documented these feature requets. In order to start the … Oct 23, 2016 · So , according to you" implementing" is wrong too. Don\'t you? I have a doubt here. I have read that … Jan 30, 2021 · Hi there, Even though i know the definitions of these verbs, i

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults
interviewer.search_tool = DuckDuckGoSearchResults()
query = websearch.search_queries[0]
list_results = interviewer.search_tool.invoke(query)

2025-07-31 02:21:21,458 - INFO - Performing web search for: latest AI-powered personal finance apps 2024
c:\Code\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
2025-07-31 02:21:21,723 - INFO - response: https://www.bing.com/search?q=latest+AI-powered+personal+finance+apps+2024&filters=ex1%3A%22ez5_19934_20299%22 200
2025-07-31 02:21:23,258 - INFO - response: https://www.bing.com/search?q=latest+AI-powered+personal+finance+apps+2024&filters=ex1%3A%22ez5_19934_20299%22&first=11&FORM=PERE 200
2025-07-31 02:21:23,370 - INFO - Web search successful.


In [22]:
list_results

'snippet: Nov 6, 2024 · Here are some of the most popular, freely available, AI-powered budgeting and financial management tools used in India. The table below compares these tools based on …, title: Top 5 AI Budgeting Tools - Analytics Vidhya, link: https://www.analyticsvidhya.com/blog/2024/11/ai-budgeting-tools/, snippet: Jun 29, 2025 · AI finance tools are mobile apps or platforms that use artificial intelligence to track, analyze, and manage your personal finances. These tools study your spending habits, give …, title: Top AI Tools for Personal Finance in India: Your Smart Money …, link: https://taxreaders.com/top-ai-tools-for-personal-finance-in-india-your-smart-money-manager-in-2025-ai-tools-for-personal-finance-in-india/, snippet: Sep 21, 2024 · Managing personal finances can often feel overwhelming. Thankfully, AI-powered tools have emerged to simplify budgeting, saving, and managing expenses., title: 10 OF BEST & COMMONLY USED AI TOOLS FOR PERSONAL …, link: https://aienabledhu

# 3. Stage 2: Generate Initial Answer

In [ ]:
# Get the first question and generate an initial answer based on the search context
question = interviewer.get_next_question()
initial_answer = interviewer.generate_initial_answer(mock_state, question, search_results)

print(f"--- Initial Answer for: '{question}' ---")
print(initial_answer)

# 4. Stage 3: Critique Answer

In [ ]:
# Critique the generated answer for weaknesses or omissions
critique = interviewer.critique_answer(mock_state, question, initial_answer)

print("--- Critique of the Answer ---")
print(critique)

# 5. Stage 4: Refine Answer

In [ ]:
# Refine the initial answer based on the critique
refined_answer = interviewer.refine_answer(mock_state, question, initial_answer, critique)

print("--- Refined Answer ---")
print(refined_answer)

# 6. Stage 5: Format for User Presentation

In [ ]:
# Format the refined answer into a user-friendly presentation
user_presentation = interviewer.present_to_user(mock_state, question, refined_answer)

print("--- Formatted for User ---")
print(user_presentation)

# 7. Full Self-Critique Workflow (All Stages Combined)

In [ ]:
# Run the entire self-critique process for a single question
print(f"Running full self-critique for question: '{question}'")
full_result = interviewer.process_question_with_self_critique(mock_state, question)

if 'error' in full_result:
    print(f"An error occurred: {full_result['error']}")
else:
    print("--- Final User Presentation from Full Workflow ---")
    print(full_result.get('user_presentation'))

# 8. Full Interview Simulation

In [ ]:
# Simulate running the agent through the first 3 questions of the interview
agent_for_run = SafeProductInterviewerAgent()
current_state = {"initial_idea": initial_idea, "qna_history": {}}

for i in range(3):
    print(f'--- Running Interview: Question {i+1} ---')
    current_state = agent_for_run.run(current_state)
    print(f'Q&A history now contains {len(current_state["qna_history"])} items.')
    print('--- End of Question ---
')

print("Full interview simulation complete.")